In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import random

In [2]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']
# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
gene_num=(df_proceed['Metadata_gene'].value_counts())
gene_num_list=gene_num.index.tolist()
# Only in list of plate is for mAP classifiation, so we filter it out
lst=['BR00116991','BR00116992','BR00116993','BR00116994','BR00117015','BR00117016','BR00117017','BR00117019','BR00116995','BR00117024','BR00117025','BR00117026','BR00117010','BR00117011','BR00117012','BR00117013']

# 8:2 Split Target Gene
random.seed(1)
train_lst=random.sample(gene_num_list, 104)#####
test_lst = []
for m in gene_num_list:
    if m not in train_lst:
        test_lst.append(m)
        
df_compound=df_proceed[df_proceed['Metadata_experiment_type']=='Compound']

# 80% target gene of Compound + Gene perturbation 

dftrain1=df_compound[df_compound['Metadata_gene'].isin(train_lst)]  

# 100% CRISPR for Train
dftrain2=df_proceed[df_proceed['Metadata_experiment_type']=='CRISPR']    

# To meet the mAP evaluation Plate
train_part_total=pd.DataFrame()
for i in lst:
    train_part_total=pd.concat([train_part_total,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
train_part0=pd.DataFrame()
for i in ['BR00116991','BR00116992','BR00116993','BR00116994']:
    train_part0=pd.concat([train_part0,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part1=pd.DataFrame()
for i in ['BR00117015','BR00117016','BR00117017','BR00117019']:
    train_part1=pd.concat([train_part1,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part2=pd.DataFrame()
for i in ['BR00116995','BR00117024','BR00117025','BR00117026']:
    train_part2=pd.concat([train_part2,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part3=pd.DataFrame()
for i in ['BR00117010','BR00117011','BR00117012','BR00117013']:
    train_part3=pd.concat([train_part3,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])  
    
# test data sampling
gene_seen=pd.DataFrame()
for i in train_lst:
    train_part0_filtered=train_part0.loc[train_part0['Metadata_gene']==i,:]
    train_part1_filtered=train_part1.loc[train_part1['Metadata_gene']==i,:]
    train_part2_filtered=train_part2.loc[train_part2['Metadata_gene']==i,:]
    train_part3_filtered=train_part3.loc[train_part3['Metadata_gene']==i,:]

# To filter seen/unseen gene from compound perturbation

    if len(train_part0_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part0_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part0_filtered[train_part0_filtered['Metadata_pert_iname']==filter_out]])
        
    if len(train_part1_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part1_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part1_filtered[train_part1_filtered['Metadata_pert_iname']==filter_out]]) 
        
    if len(train_part2_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part2_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part2_filtered[train_part2_filtered['Metadata_pert_iname']==filter_out]])  
        
    if len(train_part3_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part3_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part3_filtered[train_part3_filtered['Metadata_pert_iname']==filter_out]])

train_part_total_final=train_part_total.drop(index=list(gene_seen['Unnamed: 0'].index))
df_test1=gene_seen

# train
df_train=pd.concat([train_part_total_final,dftrain2]) 

df_test2=df_proceed[df_proceed['Metadata_gene'].isin(test_lst)]
df_test2=df_test2.drop_duplicates()
df_test2=df_test2[df_test2['Metadata_experiment_type']=='Compound']

# SEEN

# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
df_test1_rm_emptygene=df_test1[df_test1['Metadata_gene']!='empty']
y_test=df_test1_rm_emptygene[df_test1_rm_emptygene['Metadata_experiment_type']=='Compound']['Metadata_gene']
testX=df_test1_rm_emptygene[df_test1_rm_emptygene['Metadata_experiment_type']=='Compound'].drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

/tmp/ipykernel_1925246/2225210350.py:3: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, recall_score

# Scale the training and test sets using StandardScaler
scaler = StandardScaler()
trainX_scaled = scaler.fit_transform(trainX)
testX_scaled = scaler.transform(testX)

In [4]:
# Train the MLPClassifier model
mlp_clf = MLPClassifier(hidden_layer_sizes=(1280, 630, 300, 176),
                        max_iter=300, activation='relu', verbose=True, tol=1e-8,
                        solver='adam')
mlp_clf.fit(trainX_scaled, trainY)


Iteration 1, loss = 4.69253724
Iteration 2, loss = 3.90147829
Iteration 3, loss = 3.22886488
Iteration 4, loss = 2.67828407
Iteration 5, loss = 2.21154974
Iteration 6, loss = 1.80153015
Iteration 7, loss = 1.47515350
Iteration 8, loss = 1.10894353
Iteration 9, loss = 0.85199269
Iteration 10, loss = 0.67270239
Iteration 11, loss = 0.45599645
Iteration 12, loss = 0.34842806
Iteration 13, loss = 0.24566916
Iteration 14, loss = 0.17982548
Iteration 15, loss = 0.13735032
Iteration 16, loss = 0.22740864
Iteration 17, loss = 0.20972902
Iteration 18, loss = 0.17038858
Iteration 19, loss = 0.11356480
Iteration 20, loss = 0.07348475
Iteration 21, loss = 0.06519031
Iteration 22, loss = 0.07303905
Iteration 23, loss = 0.05113644
Iteration 24, loss = 0.04424915
Iteration 25, loss = 0.03986786
Iteration 26, loss = 0.02934142
Iteration 27, loss = 0.04499495
Iteration 28, loss = 0.05211017
Iteration 29, loss = 0.10521738
Iteration 30, loss = 0.10542647
Iteration 31, loss = 0.08938830
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(1280, 630, 300, 176), max_iter=300, tol=1e-08,
              verbose=True)

These below are MLP Classification

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainY_encoded = le.fit_transform(trainY)
testY_encoded = le.transform(testY)

y_pred_probs = mlp_clf.predict_proba(testX_scaled)

# Initialize a dictionary to keep track of the number of correct predictions for each label
label_counts = {label: {'total': 0, 'correct': 0} for label in set(testY)}

# Loop over each sample in the test set
for i in range(len(testY)):
    true_label = testY[i]
    true_label_encoded = testY_encoded[i]
    top_ten_labels_encoded = np.argsort(y_pred_probs[i])[::-1][:10]
    top_ten_labels = le.inverse_transform(top_ten_labels_encoded)
    if true_label in top_ten_labels:
        label_counts[true_label]['correct'] += 1
    label_counts[true_label]['total'] += 1

# Calculate the accuracy for each label and print the results
for label, counts in label_counts.items():
    if counts['total'] > 0:
        accuracy = counts['correct'] / counts['total']
        print(f"Label {label}: {accuracy:.2f} ({counts['correct']}/{counts['total']})")
    else:
        print(f"Label {label}: No samples in test set")

# Calculate the accuracy for each label and store the results in a dictionary
accuracy_dict = {}
for label, counts in label_counts.items():
    if counts['total'] > 0:
        accuracy = counts['correct'] / counts['total']
        accuracy_dict[label] = accuracy

# Create a DataFrame with the accuracy for each label
accuracy_df = pd.DataFrame.from_dict(accuracy_dict, orient='index', columns=['Accuracy'])
accuracy_df.index.name = 'Label'
accuracy_df = accuracy_df.sort_values(by='Accuracy', ascending=False)
seen_result=accuracy_df

Label ADH1C: 0.38 (6/16)
Label RNASE1: 0.69 (11/16)
Label TNNC1: 0.62 (10/16)
Label BTK: 0.62 (10/16)
Label SLCO2B1: 0.38 (6/16)
Label FFAR4: 0.50 (8/16)
Label IL1B: 0.38 (6/16)
Label PTK2B: 0.06 (1/16)
Label FPR1: 0.06 (1/16)
Label HTR2C: 0.56 (9/16)
Label CDK4: 0.50 (8/16)
Label MAPK8: 0.33 (5/15)
Label TNF: 0.62 (10/16)
Label COMT: 0.56 (9/16)
Label ALK: 0.94 (15/16)
Label S1PR2: 0.25 (4/16)
Label TGFBR1: 0.25 (4/16)
Label SSTR2: 0.38 (6/16)
Label CHRM3: 0.81 (13/16)
Label HPGDS: 0.33 (5/15)
Label SCNN1G: 0.00 (0/16)
Label DCK: 0.75 (12/16)
Label ICAM1: 0.60 (9/15)
Label PORCN: 0.73 (11/15)
Label CYP3A4: 0.47 (7/15)
Label PLA2G1B: 0.31 (5/16)
Label DHH: 0.27 (4/15)
Label P3H1: 0.53 (8/15)
Label KCNK1: 0.31 (5/16)
Label F10: 0.25 (4/16)
Label KCNH7: 0.06 (1/16)
Label RPL3: 0.75 (12/16)
Label PPARD: 0.56 (9/16)
Label MME: 0.47 (7/15)
Label RPL23A: 1.00 (16/16)
Label PRKCB: 0.56 (9/16)
Label GPR119: 0.81 (13/16)
Label DDR2: 0.50 (8/16)
Label CYP1A2: 0.31 (5/16)
Label KCTD16: 0.94 (15/1

In [6]:
#### UNSEEN
dfte22=df_test2[df_test2['Metadata_gene']!='empty']
y_test=dfte22[dfte22['Metadata_experiment_type']=='Compound']['Metadata_gene']
testX=dfte22[dfte22['Metadata_experiment_type']=='Compound'].drop('Metadata_gene', axis = 1)
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX_final=testX
testY_final=y_test
testX=np.array(testX)
testY=np.array(y_test)
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
scaler = sc.fit(trainX)
testX_scaled = scaler.transform(testX)

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainY_encoded = le.fit_transform(trainY)
testY_encoded = le.transform(testY)

y_pred_probs = mlp_clf.predict_proba(testX_scaled)

# Initialize a dictionary to keep track of the number of correct predictions for each label
label_counts = {label: {'total': 0, 'correct': 0} for label in set(testY)}

# Loop over each sample in the test set
for i in range(len(testY)):
    true_label = testY[i]
    true_label_encoded = testY_encoded[i]
    top_ten_labels_encoded = np.argsort(y_pred_probs[i])[::-1][:10]
    top_ten_labels = le.inverse_transform(top_ten_labels_encoded)
    if true_label in top_ten_labels:
        label_counts[true_label]['correct'] += 1
    label_counts[true_label]['total'] += 1

# Calculate the accuracy for each label and print the results
for label, counts in label_counts.items():
    if counts['total'] > 0:
        accuracy = counts['correct'] / counts['total']
        print(f"Label {label}: {accuracy:.2f} ({counts['correct']}/{counts['total']})")
    else:
        print(f"Label {label}: No samples in test set")

# Calculate the accuracy for each label and store the results in a dictionary
accuracy_dict = {}
for label, counts in label_counts.items():
    if counts['total'] > 0:
        accuracy = counts['correct'] / counts['total']
        accuracy_dict[label] = accuracy

# Create a DataFrame with the accuracy for each label
accuracy_df = pd.DataFrame.from_dict(accuracy_dict, orient='index', columns=['Accuracy'])
accuracy_df.index.name = 'Label'
accuracy_df = accuracy_df.sort_values(by='Accuracy', ascending=False)
unseen_result=accuracy_df

Label TUBB3: 0.69 (33/48)
Label ANXA1: 0.23 (11/47)
Label LYN: 0.73 (35/48)
Label RGS4: 0.02 (1/48)
Label ALDH2: 0.00 (0/48)
Label CA14: 0.02 (1/48)
Label CDC25A: 0.02 (1/47)
Label GUCY1B1: 0.02 (1/47)
Label HBB: 0.00 (0/48)
Label SLC7A11: 0.00 (0/47)
Label CDK9: 0.46 (22/48)
Label S1PR1: 0.00 (0/47)
Label LPAR1: 0.42 (20/48)
Label MMP2: 0.06 (3/48)
Label S1PR4: 0.00 (0/46)
Label KCNN4: 0.00 (0/47)
Label PTGIR: 0.00 (0/48)
Label AVPR1A: 0.02 (1/48)
Label TGM2: 0.04 (2/48)
Label ATM: 0.02 (1/48)
Label S100B: 0.00 (0/47)
Label CCR1: 0.00 (0/48)
Label PTGIS: 0.00 (0/47)
Label ABL1: 0.02 (1/48)
Label CACNG1: 0.09 (4/47)
Label PRKCE: 0.28 (13/46)


In [8]:
seen_result['If_seen']='seen'
unseen_result['If_seen']='unseen'

In [9]:
MLP_result=pd.concat([seen_result,unseen_result])
MLP_result=MLP_result.dropna()
print('AVG Seen ACC MLP',seen_result['Accuracy'].mean())
print('AVG Unseen ACC MLP',unseen_result['Accuracy'].mean())
print('AVG ALL ACC MLP',MLP_result['Accuracy'].mean())
#MLP_result=MLP_result.sort_values(by='Recall', ascending = False)
#MLP_result=MLP_result.iloc[:10,:]

AVG Seen ACC 0.5139851485148516
AVG Unseen ACC 0.12093503166583647
AVG ALL ACC 0.4335181954591476


In [10]:
##### Feature Transformation for Test Set Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_proceed.iloc[:,df_proceed.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_proceed.iloc[:,0:df_proceed.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_proceed['Metadata_experiment_type']], axis=1)

In [11]:
feature_new=feature_new.iloc[:,1:-1]
compound_new=feature_new[feature_new['Metadata_experiment_type']=='Compound']
compound_new.to_csv('/data/datacenter/H3C_GPU/projects/yuchen/gzsda-main/gzsda-main/data/XrayBaggage20/Compound.csv')

In [12]:
##These are used to train CCVAE
CRISPR_new=feature_new[feature_new['Metadata_experiment_type']=='CRISPR']
CRISPR_new.to_csv('/data/datacenter/H3C_GPU/projects/yuchen/gzsda-main/gzsda-main/data/XrayBaggage20/CRISPR.csv')

In [13]:
feature_new=feature_new.reset_index()
df_test1=df_test1.reset_index()
df_test2=df_test2.reset_index()
df_test1=df_test1.iloc[:,2:]
df_test2=df_test2.iloc[:,2:]
feature_new=feature_new.iloc[:,2:]

In [14]:
##### Feature Transformation Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_test1.iloc[:,df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_test1.iloc[:,0:df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_test1['Metadata_experiment_type']], axis=1)
dfte1a=feature_new

In [15]:
##### Feature Transformation Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_test2.iloc[:,df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_test2.iloc[:,0:df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_test2['Metadata_experiment_type']], axis=1)
dfte2a=feature_new

These below is for 1-NN Classification

In [16]:
##### Feature Transformation Seen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_train.iloc[:,df_train.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_train.iloc[:,0:df_train.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_train['Metadata_experiment_type']], axis=1)
dftrain_new=feature_new

# Modify the shape of data with transformed features
y_train=dftrain_new['Metadata_gene']
trainX=dftrain_new.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc(0):]
from sklearn.neighbors import KNeighborsClassifier
y_test=dfte1a['Metadata_gene']
testX=dfte1a.drop('Metadata_gene', axis = 1)
testX=testX.iloc[:,testX.columns.get_loc(0):]
testX=testX.iloc[:,:-1]

## prepare for training
trainX=trainX.iloc[:,:-1]
trainX=np.array(trainX)
trainY=y_train.to_list()
trainY=np.array(trainY)
testX=np.array(testX)
testY=np.array(y_test)
k=130
label_vectors=trainX

In [17]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

def kNCM(trainX, trainY, testX, testY, k):
    # Convert the label strings to integers
    label_to_int = {label: i for i, label in enumerate(set(trainY))}
    trainY_int = np.array([label_to_int[label] for label in trainY])
    testY_int = np.array([label_to_int[label] for label in testY])

    # Compute the mean vectors for each class in the training set
    class_means = {}
    for label in set(trainY_int):
        class_means[label] = np.mean(trainX[trainY_int == label], axis=0)

    # Use the class means as the label vectors
    label_vectors = np.array(list(class_means.values()))

    # Classify the test samples using the k-nearest class mean algorithm
    knn_clf = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    knn_clf.fit(label_vectors, list(class_means.keys()))
    nn_indices = knn_clf.kneighbors(testX, return_distance=False)

    # Compute the predicted labels for each test sample
    top_ten_labels_int = np.array([[knn_clf.classes_[i] for i in nn] for nn in nn_indices])

    # Convert the label indices to label names and filter out invalid labels
    int_to_label = {i: label for label, i in label_to_int.items()}
    top_ten_labels = np.vectorize(lambda x: int_to_label.get(x, None))(top_ten_labels_int)
    top_ten_labels = np.array([labels[labels != None][:10] for labels in top_ten_labels])

    # Merge testY and top_ten_labels into a DataFrame
    df = pd.DataFrame({'testY': testY, 'top_ten_labels': top_ten_labels.tolist()})
    lst=[]
    for i in range(len(df)):
        if df['testY'][i] in df['top_ten_labels'][i]:
            score=1
        else:
            score=0
        lst.append(score)
    df['score']=lst
    avg_df = df.groupby('testY')['score'].mean().reset_index()
    avg_df=avg_df.sort_values(by='score',ascending= False)
    return avg_df

In [18]:
KNCM_seen_result= kNCM(trainX, trainY, testX, testY, k)
KNCM_seen_result['If_seen']='seen'

In [19]:
##### Feature Transformation Unseen 
import torch
import torch.nn as nn
relu = nn.ReLU(inplace=True)
original_seen=df_train.iloc[:,df_train.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
sc=StandardScaler()
scaler = sc.fit(original_seen)
original_seen_scaled = scaler.transform(original_seen)
layer_1=np.array(relu(torch.from_numpy(np.dot(original_seen_scaled,mlp_clf.coefs_[0])+mlp_clf.intercepts_[0])))
layer_2=np.array(relu(torch.from_numpy(np.dot(layer_1,mlp_clf.coefs_[1])+mlp_clf.intercepts_[1])))
layer_3=np.array(relu(torch.from_numpy(np.dot(layer_2,mlp_clf.coefs_[2])+mlp_clf.intercepts_[2])))
transformen_seen=df_train.iloc[:,0:df_train.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
feature=pd.DataFrame(layer_3)
feature.index=transformen_seen.index
feature_new =pd.concat([transformen_seen,feature,df_train['Metadata_experiment_type']], axis=1)
dftrain_new=feature_new

##### Modify the shape and contents of data transformed new feature
y_train=dftrain_new['Metadata_gene']
trainX=dftrain_new.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc(0):]
from sklearn.neighbors import KNeighborsClassifier
y_test=dfte2a['Metadata_gene']
testX=dfte2a.drop('Metadata_gene', axis = 1)
testX=testX.iloc[:,testX.columns.get_loc(0):]
testX=testX.iloc[:,:-1]

## Prepare for training 
trainX=trainX.iloc[:,:-1]
trainX=np.array(trainX)
trainY=y_train.to_list()
trainY=np.array(trainY)
testX=np.array(testX)
testY=np.array(y_test)
k=130
label_vectors=trainX

In [20]:
KNCM_unseen_result= kNCM(trainX, trainY, testX, testY, k)
KNCM_unseen_result['If_seen']='unseen'
KNCM_total_result=pd.concat([KNCM_seen_result,KNCM_unseen_result])
print('MLP_KNCM_SEEN',KNCM_seen_result['score'].mean())
print('MLP_KNCM_UNEEN',KNCM_unseen_result['score'].mean())
print('MLP_KNCM_TOTAL',KNCM_total_result['score'].mean())

MLP_KNCM_SEEN 0.400990099009901
MLP_KNCM_UNEEN 0.1480103595910719
MLP_KNCM_TOTAL 0.3491989712548651


This is for SLPP

In [21]:
# Data pre-processing
file_path='total_new.csv'
df=pd.read_csv(file_path)
df["Metadata_gene"]=df["Metadata_gene"].fillna('empty')
df_fill_empty=df
df_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']=='negcon']
df_rm_negcon=df_fill_empty[df_fill_empty['Metadata_control_type']!='negcon']
df_rm_empty_gene=df_fill_empty[df_fill_empty['Metadata_gene']!='empty']
df_rm_orf=df_rm_empty_gene[df_rm_empty_gene["Metadata_experiment_type"]!='ORF']
# sampling
df_proceed=df_rm_orf[~df_rm_orf['Metadata_pert_type'].isin(['control'])]
gene_num=(df_proceed['Metadata_gene'].value_counts())
gene_num_list=gene_num.index.tolist()
# Only in list of plate is for mAP classifiation, so we filter it out
lst=['BR00116991','BR00116992','BR00116993','BR00116994','BR00117015','BR00117016','BR00117017','BR00117019','BR00116995','BR00117024','BR00117025','BR00117026','BR00117010','BR00117011','BR00117012','BR00117013']

# 8:2 Split Target Gene
random.seed(1)
train_lst=random.sample(gene_num_list, 104)#####
test_lst = []
for m in gene_num_list:
    if m not in train_lst:
        test_lst.append(m)
        
df_compound=df_proceed[df_proceed['Metadata_experiment_type']=='Compound']

# 80% target gene of Compound + Gene perturbation 

dftrain1=df_compound[df_compound['Metadata_gene'].isin(train_lst)]  

# 100% CRISPR for Train
dftrain2=df_proceed[df_proceed['Metadata_experiment_type']=='CRISPR']    

# To meet the mAP evaluation Plate
train_part_total=pd.DataFrame()
for i in lst:
    train_part_total=pd.concat([train_part_total,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
train_part0=pd.DataFrame()
for i in ['BR00116991','BR00116992','BR00116993','BR00116994']:
    train_part0=pd.concat([train_part0,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part1=pd.DataFrame()
for i in ['BR00117015','BR00117016','BR00117017','BR00117019']:
    train_part1=pd.concat([train_part1,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part2=pd.DataFrame()
for i in ['BR00116995','BR00117024','BR00117025','BR00117026']:
    train_part2=pd.concat([train_part2,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])
    
train_part3=pd.DataFrame()
for i in ['BR00117010','BR00117011','BR00117012','BR00117013']:
    train_part3=pd.concat([train_part3,dftrain1.loc[dftrain1['Metadata_Plate']==i,:]])  
    
# test data sampling
gene_seen=pd.DataFrame()
for i in train_lst:
    train_part0_filtered=train_part0.loc[train_part0['Metadata_gene']==i,:]
    train_part1_filtered=train_part1.loc[train_part1['Metadata_gene']==i,:]
    train_part2_filtered=train_part2.loc[train_part2['Metadata_gene']==i,:]
    train_part3_filtered=train_part3.loc[train_part3['Metadata_gene']==i,:]

# To filter seen/unseen gene from compound perturbation

    if len(train_part0_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part0_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part0_filtered[train_part0_filtered['Metadata_pert_iname']==filter_out]])
        
    if len(train_part1_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part1_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part1_filtered[train_part1_filtered['Metadata_pert_iname']==filter_out]]) 
        
    if len(train_part2_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part2_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part2_filtered[train_part2_filtered['Metadata_pert_iname']==filter_out]])  
        
    if len(train_part3_filtered['Metadata_pert_iname'].unique())==2:
        filter_out=random.choice(train_part3_filtered['Metadata_pert_iname'].unique())
        gene_seen=pd.concat([gene_seen,train_part3_filtered[train_part3_filtered['Metadata_pert_iname']==filter_out]])

train_part_total_final=train_part_total.drop(index=list(gene_seen['Unnamed: 0'].index))
df_test1=gene_seen

# train
df_train=pd.concat([train_part_total_final,dftrain2]) 

df_test2=df_proceed[df_proceed['Metadata_gene'].isin(test_lst)]
df_test2=df_test2.drop_duplicates()
df_test2=df_test2[df_test2['Metadata_experiment_type']=='Compound']

# SEEN

# Train_test pre-process
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
df_test1_rm_emptygene=df_test1[df_test1['Metadata_gene']!='empty']
y_test=df_test1_rm_emptygene[df_test1_rm_emptygene['Metadata_experiment_type']=='Compound']['Metadata_gene']
testX=df_test1_rm_emptygene[df_test1_rm_emptygene['Metadata_experiment_type']=='Compound'].drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)

/tmp/ipykernel_1925246/2225210350.py:3: DtypeWarning: Columns (2,6,7,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file_path)


In [22]:
n_components=300
sigma=1.0

In [23]:
    # SLPP Training eigenvectors
    # Standardize the data
    X_train = StandardScaler().fit_transform(trainX)
    y_train=trainY
    # Compute the pairwise distance matrix
    D = np.zeros((X_train.shape[0], X_train.shape[0]))
    for i in range(X_train.shape[0]):
        for j in range(X_train.shape[0]):
            D[i, j] = np.linalg.norm(X_train[i] - X_train[j])

    # Compute the adjacency graph using a Gaussian kernel
    W = np.zeros((X_train.shape[0], X_train.shape[0]))
    for i in range(X_train.shape[0]):
        for j in range(X_train.shape[0]):
            W[i, j] = np.exp(-D[i, j] ** 2 / (2 * sigma ** 2)) * (y_train[i] == y_train[j])

    # Compute the degree matrix
    D = np.diag(np.sum(W, axis=1))

    # Compute the Laplacian matrix
    L = D - W

    # Compute the eigenvectors
    eigvals, eigvecs = np.linalg.eig(np.dot(np.dot(X_train.T, L), X_train))

    # Sort the eigenvectors by eigenvalues in descending order
    sort_indices = np.argsort(eigvals)[::-1]
    eigvecs = eigvecs[:, sort_indices]

    # Select the first n_components eigenvectors
    eigvecs = eigvecs[:, :n_components]
    eigvecs=np.real(eigvecs)

In [24]:
# Transform training data feature  from 904 to 300 dimension
transformed_data = np.dot(df_train.iloc[:,15:], eigvecs)
new_feature=pd.DataFrame(transformed_data)
trainX1=df_train.iloc[:,:df_train.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
trainX1=trainX1.reset_index()
dftrain_trans =pd.concat([trainX1,new_feature],axis=1)

In [25]:
# Transform SEEN test data feature  from 904 to 300 dimension
transformed_data = np.dot(testX, eigvecs)
new_feature=pd.DataFrame(transformed_data)
testX1=df_test1.iloc[:,:df_test1.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
testX1=testX1.reset_index()
dfnew1 =pd.concat([testX1,new_feature],axis=1)

In [26]:
# Prepare for training KNCM
y_train=dftrain_trans['Metadata_gene']
trainX=dftrain_trans.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc(0):]
from sklearn.neighbors import KNeighborsClassifier
y_test=dfnew1['Metadata_gene']
testX=dfnew1.drop('Metadata_gene', axis = 1)
testX=testX.iloc[:,testX.columns.get_loc(0):]
trainY=y_train.tolist()
testY=y_test.tolist()

In [27]:
k=130
KNCM_seen_result= kNCM(trainX, trainY, testX, testY, k)
KNCM_seen_result['If_seen']='seen'

In [28]:
## This is for Unseen Test data preparasion
y_train=df_train['Metadata_gene']
trainX=df_train.drop('Metadata_gene', axis = 1)
df_test2_rm_emptygene=df_test2[df_test2['Metadata_gene']!='empty']
y_test=df_test2_rm_emptygene[df_test2_rm_emptygene['Metadata_experiment_type']=='Compound']['Metadata_gene']
testX=df_test2_rm_emptygene[df_test2_rm_emptygene['Metadata_experiment_type']=='Compound'].drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
testX=testX.iloc[:,testX.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y"):]
trainX=np.array(trainX)
testX=np.array(testX)
trainY=np.array(y_train)
testY=np.array(y_test)
X_train = trainX
y_train = trainY

In [29]:
## feature transformation on Unseen test set
transformed_data = np.dot(testX, eigvecs)
new_feature=pd.DataFrame(transformed_data)
testX2=df_test2.iloc[:,:df_test2.columns.get_loc("Cells_AreaShape_BoundingBoxMaximum_Y")]
testX2=testX2.reset_index()
dfnew2 =pd.concat([testX2,new_feature],axis=1)

In [30]:
## Data Prepare for training
y_train=dftrain_trans['Metadata_gene']
trainX=dftrain_trans.drop('Metadata_gene', axis = 1)
trainX=trainX.iloc[:,trainX.columns.get_loc(0):]
from sklearn.neighbors import KNeighborsClassifier
y_test=dfnew2['Metadata_gene']
testX=dfnew2.drop('Metadata_gene', axis = 1)
testX=testX.iloc[:,testX.columns.get_loc(0):]
trainY=y_train.tolist()
testY=y_test.tolist()

In [31]:
KNCM_unseen_result= kNCM(trainX, trainY, testX, testY, k)
KNCM_unseen_result['If_seen']='unseen'
KNCM_total_result=pd.concat([KNCM_seen_result,KNCM_unseen_result])
print('SLPP_KNCM_SEEN',KNCM_seen_result['score'].mean())
print('SLPP_KNCM_UNEEN',KNCM_unseen_result['score'].mean())
print('SLPP_KNCM_TOTAL',KNCM_total_result['score'].mean())

SLPP_KNCM_SEEN 0.3182343234323432
SLPP_KNCM_UNEEN 0.09082580231978936
SLPP_KNCM_TOTAL 0.2716782482439464
